<a href="https://colab.research.google.com/github/ttjh1234/CSproject_SDA/blob/main/Additional_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Additional Task

병렬처리를 위해 전처리 Process 일부 코드 수행

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import zipfile
import os
import tensorflow as tf
from tensorflow import keras
import re

In [ ]:
def making_pose_input(df):
  final_input=[]
  if ((df.iloc[8][0]!= None) & (df.iloc[8][1]!=None))&((df.iloc[11][0] != None) & (df.iloc[11][1] != None)):
    hip=((df.iloc[8][0]+df.iloc[11][0])/2,(df.iloc[8][1]+df.iloc[11][1])/2)
    final_input.append(np.float32(hip))
  else:
    final_input.append((np.nan,np.nan))
  right_hip=df.iloc[8]
  final_input.append(np.float32(right_hip))
  right_knee=df.iloc[9]
  final_input.append(np.float32(right_knee))
  right_foot=df.iloc[10]
  final_input.append(np.float32(right_foot))
  left_hip=df.iloc[11]
  final_input.append(np.float32(left_hip))
  left_knee=df.iloc[12]
  final_input.append(np.float32(left_knee))
  left_foot=df.iloc[13]
  final_input.append(np.float32(left_foot))
  spine=df.iloc[1]
  final_input.append(np.float32(spine))
  if ((df.iloc[1][0] != None)&(df.iloc[1][1] != None))&((df.iloc[0][0] != None)&(df.iloc[0][1] != None)):
    thorax=(df.iloc[1][0]*2-(df.iloc[0][0]+df.iloc[1][0])/2,df.iloc[1][1]*2-(df.iloc[0][1]+df.iloc[1][1])/2)
    final_input.append(np.float32(thorax))
  else:
    final_input.append((np.nan,np.nan))
  head=df.iloc[0]
  final_input.append(np.float32(head))
  left_shoulder=df.iloc[5]
  final_input.append(np.float32(left_shoulder))
  left_elbow=df.iloc[6]
  final_input.append(np.float32(left_elbow))
  left_wrist=df.iloc[7]
  final_input.append(np.float32(left_wrist))
  right_shoulder=df.iloc[2]
  final_input.append(np.float32(right_shoulder))
  right_elbow=df.iloc[3]
  final_input.append(np.float32(right_elbow))
  right_wrist=df.iloc[4]
  final_input.append(np.float32(right_wrist))
  return final_input

In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/saved_model/my_model')

In [ ]:
# 모델의 입력
# 나중에 사용할 수도 있는 label 열도 일단은 생성.
def data2d_import(path):
  # 이미지 읽어오기
  image = cv2.imread(path,cv2.IMREAD_COLOR)
  image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image2 = cv2.resize(image, dsize=(368, 368), interpolation=cv2.INTER_AREA)
  image3=np.transpose(np.float32(image2[:,:,:,np.newaxis]), (3,0,1,2))
  return image3

def data_to_skeleton(image):
  output=new_model.predict(image)
  H = output[11].shape[1]
  W = output[11].shape[2]
  conf=output[11]
  imageHeight, imageWidth, _ = (368,368,3)
  points = []
  for i in range(0,18):
      # 해당 신체부위 신뢰도 얻음.
      probMap = conf[0, :, :, i]
  
      # global 최대값 찾기
      minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

      # 원래 이미지에 맞게 점 위치 변경
      x = (imageWidth * point[0]) / W
      y = (imageHeight * point[1]) / H     
      points.append((int(x), int(y)))
  return points

def data_to_input(data):
  temp=pd.DataFrame(columns=['x','y'])
  x_list=[]
  y_list=[]
  for i in data:
      x_list.append(i[0])
      y_list.append(i[1])
  temp['x']=x_list
  temp['y']=y_list
  list_temp=making_pose_input(temp)
  a=pd.DataFrame(list_temp,columns=['x','y'])
  return a

In [ ]:
# 입력 데이터 전처리 함수
def input_making(path):
  image=data2d_import(path)
  points=data_to_skeleton(image)
  result=data_to_input(points)
  return result 

In [ ]:
main_path="/content/drive/MyDrive/cv_data/"

In [ ]:
os.listdir(main_path)[14:20]

['P028', 'P022', 'P021', 'P030', 'P024', 'P026']

In [ ]:
main_path="/content/drive/MyDrive/cv_data/"
for i in os.listdir(main_path)[5:6]:
  sub_path1=main_path+i+'/'
  for j in os.listdir(sub_path1):
    if int(j[2:4])==53:
      print(j)

A053_P006_G001_C001
A053_P006_G001_C002
A053_P006_G001_C003
A053_P006_G001_C004
A053_P006_G001_C005
A053_P006_G001_C006
A053_P006_G001_C007
A053_P006_G001_C008
A053_P006_G002_C001
A053_P006_G002_C002
A053_P006_G002_C003
A053_P006_G002_C004
A053_P006_G002_C005
A053_P006_G002_C006
A053_P006_G002_C007
A053_P006_G002_C008
A053_P006_G003_C001
A053_P006_G003_C002
A053_P006_G003_C003
A053_P006_G003_C004
A053_P006_G003_C005
A053_P006_G003_C006
A053_P006_G003_C007
A053_P006_G003_C008


In [ ]:
main_path="/content/drive/MyDrive/cv_data/"
for i in os.listdir(main_path)[14:20]:
  sub_path1=main_path+i+'/'
  for j in os.listdir(sub_path1):
    if int(j[2:4]) not in [44,45,46,47,48,49,50,51,53]:
      sub_path2=sub_path1+j+'/'
      temp=pd.DataFrame(columns=['x','y'])
      if j in os.listdir("/content/drive/MyDrive/2d_skeleton/"+i+"/"):
        print("이미 처리한 파일입니다.")
        continue 
      for k in range(30):
        filepath=sub_path2+str(k)+'.jpg'
        print(filepath)
        x_temp = input_making(filepath)
        temp=pd.concat([temp,x_temp],axis=0)
      write_path="/content/drive/MyDrive/2d_skeleton/"+i+"/"+j
      try:
        os.mkdir(write_path)
        temp.to_csv(write_path+'/'+j+'.csv',index=False)
      except:
        print("이미 생성된 폴더입니다.\n")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다